In [19]:
import json
import requests as rq
import os
import time
import pandas as pd
import numpy as np
import requests as rq

In [63]:
presence_with_fungi = pd.read_csv("../results/idigbio-sample/input/presence.tsv", sep="\t")
presence = presence_with_fungi[presence_with_fungi["kingdom"] != "fungi"]
len(presence)

12133

In [3]:
presence.iloc[0]

kingdom                    plantae
phylum                 dicotyledon
family                  asteraceae
genus                      Acmella
specificepithet            ciliata
country                     Brasil
stateprovince                 Acre
county             Cruzeiro Do Sul
Name: 300, dtype: object

In [28]:
import regex as re
re.sub(r'[^\w]', '', "ahahah[]aha")

'ahahahaha'

In [61]:
presence[presence["county"] == "Rockland"]

,kingdom,phylum,family,genus,specificepithet,country,stateprovince,county


In [38]:
def trim_county_name(county):
    parts = county.split()
    if len(parts) > 1 and re.sub(r'[^\w]', '', parts[-1]).lower() in ("co", "county", "mun", "par", "prov"):
        return " ".join(parts[:-1])
    else:
        return county

In [39]:
def make_record_count_query(record):
    search_values = record.astype(str).to_dict()
    if "county" in search_values:
        search_values["county"] = {
            "type": "prefix",
            "value": trim_county_name(search_values["county"])
        }
    return {
        "rq": search_values,
        "limit": 1,
        "offset": 0
    }

In [45]:
def get_record_count(record):
    query = make_record_count_query(record)
    return rq.post("http://search.idigbio.org/v2/search/records/", json=query).json()["itemCount"]

In [64]:
group_by = ["country", "stateprovince", "county"]
df = presence_with_fungi[group_by].drop_duplicates()
df["county"] = df["county"].map(trim_county_name)
df.drop_duplicates()
df["record count"] = df.apply(axis=1, func=get_record_count)
counties = df

In [65]:
counties.to_csv("../results/idigbio-sample/input/record-counts-by-county.tsv", sep="\t", index=False)

In [66]:
group_by = ["country", "stateprovince"]
df = presence_with_fungi[group_by].drop_duplicates()
df["record count"] = df.apply(axis=1, func=get_record_count)
stateprovinces = df

In [67]:
stateprovinces.to_csv("../results/idigbio-sample/input/record-counts-by-stateprovince.tsv", sep="\t", index=False)

In [10]:
group_by = ["kingdom", "phylum", "family", "genus", "specificepithet"]
df = presence[group_by].drop_duplicates()
df["record count"] = df.apply(axis=1, func=get_record_count)
species = df

In [11]:
species.to_csv("../results/idigbio-sample/input/record-counts-by-species.tsv", sep="\t", index=False)

In [12]:
group_by = ["kingdom", "phylum", "family", "genus"]
df = presence[group_by].drop_duplicates()
df["record count"] = df.apply(axis=1, func=get_record_count)
genera = df

In [13]:
genera.to_csv("../results/idigbio-sample/input/record-counts-by-genus.tsv", sep="\t", index=False)

In [14]:
group_by = ["kingdom", "phylum", "family"]
df = presence[group_by].drop_duplicates()
df["record count"] = df.apply(axis=1, func=get_record_count)
families = df

In [15]:
families.to_csv("../results/idigbio-sample/input/record-counts-by-family.tsv", sep="\t", index=False)

In [16]:
group_by = ["kingdom", "phylum"]
df = presence[group_by].drop_duplicates()
df["record count"] = df.apply(axis=1, func=get_record_count)
phyla = df

In [17]:
phyla.to_csv("../results/idigbio-sample/input/record-counts-by-phylum.tsv", sep="\t", index=False)